## Dependencias

In [26]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_blobs,load_iris

from sklearn.linear_model import Perceptron,PassiveAggressiveClassifier
from sklearn.svm import SVC 
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.pipeline import make_pipeline

from sklearn.model_selection import train_test_split,RandomizedSearchCV  
from sklearn.metrics import accuracy_score,confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns
import cufflinks as cf

from functools import reduce 

cf.go_offline()
pd.set_option('display.max_columns', None)   
pd.set_option('display.float_format', lambda x: '%.3f' % x)

## Lectura de datos

In [27]:
data = pd.read_excel('responses.xlsx')

In [28]:
varc = ['¿Cuántos años tienes? 🎂',
        '¿Qué tanto evitaste que tu carrera tuviera matemáticas?',
        '¿Qué promedio obtuviste en el bachillerato?',
        'Si tuvieras que volver a clases presenciales ¿Cuánto tiempo tardarías en minutos en llegar a la FES Acatlán? ⏱',
        '¿Cuánto gastarías de pasaje / gasolina a la semana?',
        '¿A cuánto asciende el ingreso mensual neto de todas las personas que trabajan y viven contigo?',
        'Hazle un test a tu Internet (ingresando a speedtest.net) y compártenos su velocidad'
        ]
vart = '¿A qué carrera ingresaste?'
X = data[varc]
y = data[vart]
names = ['edad',
                                'evitar_mate',
                                'promedio',
                                'tiempo_llegada',
                                'gasto_semanal',
                                'ingreso_mensual',
                                'velocidad_internet']
X.rename(columns=dict(zip(varc,names)),inplace=True)

In [29]:
sc = MinMaxScaler()

In [30]:
X = sc.fit_transform(X)

In [31]:
y = (y.isin(['Actuaría','Matemáticas Aplicadas y Computación','Ingeniería Civil'])).astype(int)

In [32]:
y.value_counts(1)

¿A qué carrera ingresaste?
0   0.643
1   0.357
Name: proportion, dtype: float64

## Partición

In [33]:
Xt,Xv,yt,yv = train_test_split(X,y,test_size=0.3)

In [34]:
# Xt= Xt.to_numpy()
# Xv = Xv.to_numpy()
yt = yt.to_numpy()
yv = yv.to_numpy()

In [35]:
Xt.shape,Xv.shape,yt.shape,yv.shape

((1468, 7), (630, 7), (1468,), (630,))

## Aprendizaje

### Perceptrón

In [36]:
per = Perceptron()
per.fit(Xt,yt)

Perceptron()

#### Vector de parámetros $\vec{\theta}$ y sesgo ${\theta_0}$

In [38]:
dict(zip(names,per.coef_[0])),per.intercept_

({'edad': -1.8461538461538616,
  'evitar_mate': -2.0,
  'promedio': -0.894285714285691,
  'tiempo_llegada': -0.2129166666666647,
  'gasto_semanal': -0.6017500000000031,
  'ingreso_mensual': 1.5432350000000086,
  'velocidad_internet': 0.6207113333333352},
 array([1.]))

In [39]:
accuracy_score(yt,per.predict(Xt)),accuracy_score(yv,per.predict(Xv))

(0.7622615803814714, 0.765079365079365)

In [40]:
confusion_matrix(yt,per.predict(Xt)),confusion_matrix(yv,per.predict(Xv))

(array([[776, 145],
        [204, 343]]),
 array([[352,  77],
        [ 71, 130]]))

## Pasivo-Agresivo

### Híper-Parametrización

In [41]:
hp = {'C':np.arange(0.01,2,0.01)}
print(reduce(lambda x,y:x*y,[len(hp[k]) for k in hp]))

199


In [42]:
pa = PassiveAggressiveClassifier()

In [44]:
grid = RandomizedSearchCV(estimator=pa,param_distributions=hp,cv=5,scoring='accuracy',n_jobs=-1,n_iter=50)
grid.fit(Xt,yt)

RandomizedSearchCV(cv=5, estimator=PassiveAggressiveClassifier(), n_iter=50,
                   n_jobs=-1,
                   param_distributions={'C': array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.4...
       1.33, 1.34, 1.35, 1.36, 1.37, 1.38, 1.39, 1.4 , 1.41, 1.42, 1.43,
       1.44, 1.45, 1.46, 1.47, 1.48, 1.49, 1.5 , 1.51, 1.52, 1.53, 1.54,
       1.55, 1.56, 1.57, 1.58, 1.59, 1.6 , 1.61, 1.62, 1.63, 1.64, 1.65,
       1.66, 1.67, 1.68, 1.69, 1.7 , 1.71, 1.72, 1.73, 1.74, 1.75, 1.76,
       1.77, 1.78, 1.79, 1.8 , 1.81, 1.82, 1.83, 1.84, 1.85, 1.86, 1.87,
       1.88, 1.89, 1.9 , 1.91, 1.92, 1.93, 1.94, 1.95, 1.96, 1.97, 1.98,
       1.99])},
                   scoring='accuracy')

In [45]:
grid.best_params_,grid.best_score_

({'C': 0.33}, 0.7792923312669777)

### Estimación de parámetros 

In [46]:
pa = grid.best_estimator_
pa.fit(Xt,yt)

PassiveAggressiveClassifier(C=0.33)

### Vector de parámetros $\vec{\theta}$ y sesgo ${\theta_0}$

In [48]:
dict(zip(names,pa.coef_[0])),pa.intercept_

({'edad': -0.3764405732073235,
  'evitar_mate': -4.633842400020679,
  'promedio': -1.6093733279249718,
  'tiempo_llegada': -0.5956002611086091,
  'gasto_semanal': -0.6816674621236611,
  'ingreso_mensual': 2.045018233003498,
  'velocidad_internet': 1.6611744020430113},
 array([1.41461719]))

In [49]:
accuracy_score(yt,pa.predict(Xt)),accuracy_score(yv,pa.predict(Xv))

(0.7656675749318801, 0.7619047619047619)

In [50]:
confusion_matrix(yt,pa.predict(Xt)),confusion_matrix(yv,pa.predict(Xv))

(array([[787, 134],
        [210, 337]]),
 array([[358,  71],
        [ 79, 122]]))

## Máquina Vector Soporte

### Híper-Parametrización

In [51]:
hp =  {'C': np.arange(0.1,2,0.1), 'kernel': ['linear', 'rbf', 'poly'],
         'degree':np.arange(1,5,1)
       }
print(reduce(lambda x,y:x*y,[len(hp[k]) for k in hp]))

228


In [52]:
svm = SVC()

In [54]:
grid = RandomizedSearchCV(estimator=svm,
                          param_distributions=hp,
                          cv=5,
                          scoring='accuracy',
                          n_jobs=-1,
                          n_iter=50)
grid.fit(Xt,yt)

RandomizedSearchCV(cv=5, estimator=SVC(), n_iter=50, n_jobs=-1,
                   param_distributions={'C': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9]),
                                        'degree': array([1, 2, 3, 4]),
                                        'kernel': ['linear', 'rbf', 'poly']},
                   scoring='accuracy')

In [55]:
grid.best_params_,grid.best_score_

({'kernel': 'linear', 'degree': 2, 'C': 0.5}, 0.7874788140512178)

### Estimación de parámetros 

In [56]:
svm = grid.best_estimator_
svm.fit(Xt,yt)

SVC(C=0.5, degree=2, kernel='linear')

In [57]:
dict(zip(names,svm.coef_[0])),svm.intercept_

({'edad': -0.0018699662176780985,
  'evitar_mate': -4.0041448572768985,
  'promedio': -0.005352517463587003,
  'tiempo_llegada': -0.005660301504128884,
  'gasto_semanal': -0.0031297159006790382,
  'ingreso_mensual': 0.01016923850041375,
  'velocidad_internet': 0.7926334903626925},
 array([1.00253992]))

### Vector de parámetros $\vec{\theta}$ y sesgo ${\theta_0}$

In [58]:
accuracy_score(yt,svm.predict(Xt)),accuracy_score(yv,svm.predict(Xv))

(0.7874659400544959, 0.7619047619047619)

In [59]:
confusion_matrix(yt,svm.predict(Xt)),confusion_matrix(yv,svm.predict(Xv))

(array([[724, 197],
        [115, 432]]),
 array([[329, 100],
        [ 50, 151]]))

## Prueba

In [62]:
names

['edad',
 'evitar_mate',
 'promedio',
 'tiempo_llegada',
 'gasto_semanal',
 'ingreso_mensual',
 'velocidad_internet']

In [91]:
profe= np.array([17,1,9.5,80,250,10000,10]).reshape(1, -1)

In [92]:
profe.shape

(1, 7)

In [93]:
svm.predict(sc.transform(profe))

/home/jose/entornos/tse/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names



array([1])

In [100]:
svm.predict(X[2096].reshape(1,-1))

array([0])

In [98]:
data.loc[data.index==2096]

,Timestamp,Selecciona tu sexo 👤,¿Cuántos años tienes? 🎂,¿Cuál es tu situación sentimental? ❤️,¿Trabajas? 💼,¿Tienes hijos? 🍼,¿Cuál es tu código postal?,¿A qué carrera ingresaste?,¿En qué grupo estás?,¿Qué tanto evitaste que tu carrera tuviera matemáticas?,Esta carrera ¿Fue tu primera opción?,¿La FES Acatlán fue tu primera opción para cursar esta carrera?,¿De qué bachillerato provienes?,¿Qué promedio obtuviste en el bachillerato?,Si tuvieras que volver a clases presenciales ¿Cuánto tiempo tardarías en minutos en llegar a la FES Acatlán? ⏱,¿Cuánto gastarías de pasaje / gasolina a la semana?,¿A cuánto asciende el ingreso mensual neto de todas las personas que trabajan y viven contigo?,¿En dónde tomas tus clases en línea?,¿Cuántas personas viven contigo?,De las personas que viven contigo ¿Cuántas toman clases en línea?,De las personas que viven contigo ¿Cuántas trabajan en línea?,¿Cuántas computadoras hay en tu casa?,¿Cómo funciona tu internet para las clases en línea?,Hazle un test a tu Internet (ingresando a speedtest.net) y compártenos su velocidad,¿Qué sistema operativo tiene la computadora que utilizas para estudiar en línea?,¿Con cuántas personas compartes habitación mientras tomas clases en línea?,Aproximadamente ¿Cuánto has gastado para adaptarte a las clases en línea?,Del 1 al 5 ¿Qué tan bien estaban preparados tus profesores para dar clases en línea?,¿Cuánto tiempo en minutos dedicas a estudiar fuera de las clases en línea?,Aproximadamente ¿Cuántas veces revisas tu celular mientras estás en una clase en línea?,Del 1 al 5 ¿Cuánto extrañas las clases presenciales? 📖,Del 1 al 5 ¿Cuánto extrañas convivir con tus compañeros o amigos de la escuela? 👩🏻‍🤝‍🧑🏼,Del 1 al 5 ¿Cuánto extrañas las actividades extras que ofrecía tu escuela? ⚽🎹🎬,Del 1 al 5 ¿Cuánto te agrada poner la cámara en las clases en línea?,Del 1 al 5 ¿Qué tan buen estudiante en línea consideras que eres?,¿Te gustaría volver a clases presenciales?
2096,2021-10-22 11:39:59.248,Mujer,19,Soltero,No,No,54910,Derecho Escolarizado,1105,2,No,Sí,Otro,8.600,120.000,300.000,15500,Mi casa,4,2,0,3,3,5.400,Windows,0,0.000,4,120,3,5,3,2,3,2,Si
